### Problem Set 5

### Olivia Natan

We are working again with the Brock & Mirman model, which we will estimate this time via GMM and SMM.

Recall that our model is (with a slight change of assumption on $\epsilon$): 

$$ (c_t)^{-1} - \beta E[r_{t+1} c_{t+1}^{-1}]=0$$

$$c_t + k_{t+1} − w_t − r_t k_t = 0 $$

$$w_t - (1-\alpha)exp(z_t) k_t^{\alpha}=0$$

$$r_t - \alpha exp(z_t) k_t^{\alpha-1}=0$$

$$z_t = \rho z_{t-1} + (1-\rho) \mu + \epsilon_t$$

$$where \;\; E[\epsilon_t]=0 $$

We want to estimate $(\alpha, \beta, \rho, \mu, \sigma)$ where  $\alpha ,\beta \in (0,1)$, $\mu, \sigma >0$ and $\rho \in (-1, 1)$.

The data we have is 100 quarters of aggregate measures of $c_t$, consumption, $k_{t+1}$, savings from period t, $r_t$, the interest rate, and the wage per unit labor $w_t$.

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as mpl
import seaborn as sns
import scipy.stats as sts
np.random.seed(200)
%matplotlib inline

In [3]:
df = pd.read_csv('MacroSeries.txt',sep=',',header=None,names=['ct','kt','wt','rt'])
df['tt'] = df.index+1
df.head()

,ct,kt,wt,rt,tt
0,1.067109e+07,8.040697e+06,1.059438e+07,0.954120,1
1,1.169487e+07,7.595072e+06,1.161079e+07,1.107009,2
2,9.292394e+06,8.323735e+06,9.225588e+06,0.802596,3
3,7.913165e+06,6.613792e+06,7.856274e+06,0.860176,4
4,8.140994e+06,5.632136e+06,8.082466e+06,1.039183,5


## Problem 1 : Estimate via GMM

### Part A: we want to recover estimates of $\alpha$, $\beta$, $\rho$, and $\mu$ via GMM

We want to estimate $\alpha$ and $\beta$.
We use the following 2 moment conditions based on the Euler eqn:

$$E\left[\beta \alpha exp(z_{t+1})k_{t+1}^{1-\alpha} \frac{c_t}{c_{t+1}}-1\right]=0$$

$$E\left[(\beta  \alpha exp(z_{t+1})k_{t+1}^{1-\alpha} \frac{c_t}{c_{t+1}}-1)w_t\right]=0$$

We will use the unconditional moment restriction: $E[\epsilon_t]=0$. We use data on $r_t$ and $k_t$ and equation (4) to back
out a time series for $z_t$. 

In particular, note that we can rewrite the equation by moving terms around and taking logs to show: $$log\left(\frac{r_t}{\alpha k_t^{\alpha-1}}\right) = z_t$$


Then note that our unconditional moment restriction also implies that $E[\epsilon_{t+1}] = 0$, which in turn implies that $E[\epsilon_{t+1}\epsilon_t] = 0$.

This allows us to estimate $\rho$ and $\mu$.

Specifically we can take the following 2 moment restrictions to the data (noting we plug in the above equivalence for $z_t$ as needed:

$$E\left[z_{t+1} - \rho z_t - (1-\rho)\mu \right]=0$$

$$E\left[(z_{t+1} - \rho z_t - (1-\rho)\mu )z_t\right]=0$$

In [4]:
# Define a function that returns z given r, k, alpha
def get_zt(rs,ks,alpha) :
    #if ((alpha>1) or (alpha<0)):
    #    print('invalid value for alpha')
    #    return
    # model-based function to return a scalar or vector given 2 vector and one scalar inputs
    zt = np.log(rs/(alpha*(ks**(1-alpha))))
    return zt

In [5]:
def errors_A(params,*args):
    alpha, beta, rho, mu = params
    rs,ks,cs,ws = args
    zt = get_zt(rs,ks,alpha)
    cplus1 = cs[1:]
    kplus1 = ks[1:]
    rplus1 = rs[1:]
    rs = rs[:len(cs)-1]
    ks = ks[:len(ks)-1]
    cs = cs[:len(cs)-1]
    ws = ws[:len(ws)-1] # shorten so we have 99 rows
    ztplus1 = zt[1:]
    zt = zt[0:len(zt)-1]
    mom1 = ztplus1 - rho*zt - (1-rho)*mu
    mom2 = mom1*zt
    mom3= beta*alpha*np.exp(ztplus1)*(kplus1**(1-alpha))*cs*(cplus1**-1) -1
    mom4= (beta*alpha*np.exp(ztplus1)*(kplus1**(1-alpha))*cs*(cplus1**-1) -1)*ws
    errors = np.array([np.mean(mom1),np.mean(mom2),np.mean(mom3),np.mean(mom4)])
    return errors

In [6]:
# define our weighting matrix as the identity
def crit_A(params, *args):
    alpha,beta,rho,mu= params
    rs,ks,cs,ws,W=args
    errvec = errors_A([alpha,beta,rho,mu],rs,ks,cs,ws)
    objective = np.dot(errvec.T,np.dot(W,errvec))
    return objective

In [9]:
crit_A([0.5,0.95,0.7,9],df['rt'],df['kt'],df['ct'],df['wt'],np.eye(4))

80288661730.636505

In [10]:
import scipy.optimize as opt
GMM_A_initvals = [0.3,0.9,0.4,0.3]
W_init = np.eye(4)
GMM_A_args = (df['rt'],df['kt'],df['ct'],df['wt'],W_init)
bounds_A = ((1e-10,1-1e-10),(1e-10,1-1e-10),(-1+1e-10,1-1e-10),(1e-10,None))
#results_A = opt.minimize(crit_A,GMM_A_initvals,args=(GMM_A_args),method='L-BFGS-B',bounds=bounds_A,tol=1e-12,options={'eps':1e-5})
results_A = opt.basinhopping(crit_A,GMM_A_initvals,minimizer_kwargs=dict(args=(GMM_A_args),method='L-BFGS-B',bounds=bounds_A,tol=1e-12),niter=1000)
results_A

C:\Users\Olivia\Anaconda\lib\site-packages\scipy\optimize\_basinhopping.py:284: RuntimeWarning: overflow encountered in exp
  w = min(1.0, np.exp(-(energy_new - energy_old) * self.beta))


                        fun: 3.8988236249360624e-05
 lowest_optimization_result:       fun: 3.8988236249360624e-05
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([  9.44244283e-05,   1.12743816e+06,   1.35139231e-04,
         0.00000000e+00])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 270
      nit: 24
   status: 0
  success: True
        x: array([ 0.31029036,  0.97657231,  1.        ,  3.70565628])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 257
                       nfev: 204545
                        nit: 1000
                          x: array([ 0.31029036,  0.97657231,  1.        ,  3.70565628])

In [17]:
results_A.lowest_optimization_result.hess_inv.todense()

array([[  4.11165222e+05,  -1.30129212e+01,  -1.25299115e+06,
         -1.39868174e+04],
       [ -1.30129212e+01,   4.11845445e-04,   3.96558683e+01,
          4.42668264e-01],
       [ -1.25299115e+06,   3.96558683e+01,   3.81839330e+06,
          4.26237438e+04],
       [ -1.39868174e+04,   4.42668264e-01,   4.26237438e+04,
          4.76797872e+02]])

## Problem 2: Estimate Via SMM

We can take the following analytical solution to allow us to simulate data: 

$$k_{t+1} = \alpha \beta exp(z_t)k_t^{\alpha}$$

With this, we need a few initial conditions to do the simulation. First, let $z_1=\mu$, and let $k_{1,sim} = mean(k_t)$ from the data. These are the same across simulations.

What varies across simulations is our draws for $\epsilon \sim N(0,\sigma)$. Thus the parameters we are estimating are $(\alpha, \beta, \rho, \mu, \sigma)$. 

With our draws, for each simulation, we can compute the series of productivity shocks. With those, and the initial condition for capital, we can forward solve for each period's savings. 

With these, we can use model equations to solve for $w_t$ and $r_t$ based on the model given parameters, then use the budget constrain to back out consumption series.

The moments we will be considering are as follows: 

1) $mean(c_t)$

2) $mean(k_t)$

3) $var(c_t)$

4) $var(k_t)$

5) $corr(c_t, k_t)$

6) $corr(k_t, k_{t+1})$

For the simulations, we let $T=100$ and $S=1000$. We also specify the bounds as follows: $\alpha,\beta \in [0.01,0.99]$, $\rho \in [-0.99,0.99]$, $\mu \in [-0.5,1]$, and $\sigma \in [0.001,1]$. We use the identity as the weighting matrix.

In [85]:
def sim_zt(rho,mu,nus,sigma):
    #nus are 100 deviates from normal 0,1 stored outside optimization. We can just multiply by sqrt(sigma)
    zt = np.zeros(shape=(100,1000))
    epsilons = nus*(sigma**0.5)
    zt[0] = mu
    for i in range(1,100):
        zt[i] = rho*zt[i-1] + (1-rho)*mu + epsilons.T[i]
    return zt

In [82]:
def sim_kt(alpha,beta,zts,k1):
    # k1 set from mean of data
    kt = np.zeros(shape=(100,1000))
    kt[0] = k1
    for i in range(1,100):
        kt[i] = alpha*beta*np.exp(zts[i-1])*(kt[i-1]**alpha)
    return kt

In [20]:
def sim_wt(alpha,zts,kts):
    wt = (1-alpha)*np.exp(zts)*(kts**alpha)
    return wt

def sim_rt(alpha,zts,kts):
    rt = alpha*np.exp(zts)*(kts**(1-alpha))
    return rt

In [34]:
def sim_ct(kts,wts,rts): #$$c_t + k_{t+1} − w_t − r_t k_t = 0 $$
    # can only get 99 ct's this way (unless we assume k_101=0 )
    ktplus1 = np.roll(kts,-1)
    ktplus1[99] = 0
    ct = wts + rts*kts - ktplus1
    return ct

In [99]:
def get_moments(cts,kts):
    if cts.ndim ==1:
        mean_ct = cts.mean()
        mean_kt = kts.mean()
        var_ct = cts.var()
        var_kt = kts.var()
        corr_ct_kt = np.corrcoef(cts,kts)[0][1]
        ktplus1 = kts[1:]
        ktshort = kts[:99]
        corr_kt_kt1 = np.corrcoef(ktshort,ktplus1)[0][1]
    if cts.ndim==2:
        mean_ct =cts.mean()
        mean_kt = kts.mean()
        var_ct = cts.var(axis=1).mean()
        var_kt = kts.var(axis=1).mean()
        indiv_corrs = np.zeros(shape=(cts.shape[0],2))
        for i in range(0,cts.shape[0]):
            indiv_corrs[i][0] = np.corrcoef(cts[i],kts[i])[0][1]
            ktplus1 = kts[i][1:]
            ktshort = kts[i][:99]
            indiv_corrs[i][1] = np.corrcoef(ktshort,ktplus1)[0][1]
        corr_ct_kt, corr_kt_kt1 = indiv_corrs.mean(axis=0)
    return np.array([mean_ct,mean_kt,var_ct,var_kt,corr_ct_kt,corr_kt_kt1])

In [93]:
def sim_data(params,*args):
    ## parameters are 5 model parameters
    alpha,beta,rho,mu,sigma = params
    nus,kmean = args
    zts = sim_zt(rho,mu,nus,sigma)
    kts = sim_kt(alpha,beta,zts,kmean)
    wts = sim_wt(alpha,zts,kts)
    rts = sim_rt(alpha,zts,kts)
    cts = sim_ct(kts,wts,rts)
    return np.array([cts.T,kts.T])

In [ ]:
    #for j in range(1,S):
    #    zts = np.append(zts,sim_zt(rho,mu,nus[j],sigma),axis=1)
    #    print(zts.shape)
    #    kts = np.append(kts,sim_kt(alpha,beta,zts[j],kmean))
    #    wts = np.append(wts,sim_wt(alpha,zts[j],kts[j]))
    #    rts = np.append(rts,sim_rt(alpha,zts[j],kts[j]))
    #    cts = np.append(cts,sim_ct(kts[j],wts[j],rts[j]))

In [103]:
get_moments(sim_data([0.5,0.9,0.6,0.5,0.5],nus,df['kt'].mean())[0],sim_data([0.5,0.9,0.6,0.5,0.5],nus,df['kt'].mean())[1])-get_moments(df['ct'],df['kt'])

array([  1.57796931e+08,  -7.39779692e+06,   2.80471840e+18,
        -2.29219881e+12,   1.20975111e-01,   1.22400595e-01])

In [111]:
def error_vec(params, nus,kmean, simple):
    alpha,beta,rho,mu,sigma = params
    simulated_data = sim_data([alpha,beta,rho,mu,sigma],nus,kmean)
    sim_mom = get_moments(simulated_data[0],simulated_data[1])
    data_mom=get_moments(df['ct'],df['kt'])
    if simple:
        errors = sim_mom - data_mom 
    else:
        errors = (sim_mom - data_mom)/data_mom
    return errors

def crit_B(params,*args):
    alpha,beta,rho,mu,sigma = params
    nus,kmean,W = args
    errors = error_vec([alpha,beta,rho,mu,sigma],nus,kmean,simple=False)
    crit = np.dot(np.dot(errors.T, W), errors)
    #print(crit)
    return crit

In [105]:
# Now we want to generate our simulated nu draws outside data
nus = sts.norm.rvs(size=(1000*100)).reshape(1000,100) # 100 draws for each simulation
# now we can call each element as the draws for each simulation
k1 = df['kt'].mean()

In [112]:
import scipy.optimize as opt
SMM_init_vals = [0.8,  0.5, 0,  0.9, 0.01]
W_init = np.eye(6)
SMM_args = (nus,k1,W_init)
bounds_SMM = ((0.01,0.99),(0.01,0.99),(-0.99,0.99),(-0.5,1),(0.001,1))
results_SMM = opt.minimize(crit_B,SMM_init_vals, args=(SMM_args),method='L-BFGS-B',bounds=bounds_SMM,tol=1e-12)
results_SMM

      fun: 0.2440120369711736
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([  2.36391158e+00,   2.22532676e-01,   6.71407374e-05,
         1.38750164e-01,   4.71379824e+01])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 738
      nit: 34
   status: 0
  success: True
        x: array([ 0.98041205,  0.98492512, -0.65024428,  0.35622345,  0.001     ])

In [115]:
print("Errors at Optimum: ",error_vec(results_SMM.x,nus,k1,simple=True))
print("Data Moments",get_moments(df['ct'],df['kt']))

('Errors at Optimum: ', array([ -5.40709163e+05,   2.98406682e+06,   4.91658762e+11,
        -3.92689023e+11,  -1.93350142e-01,   7.35129325e-02]))
('Data Moments', array([  1.05208478e+07,   7.47254456e+06,   5.69942826e+12,
         2.84500086e+12,   8.79024854e-01,   8.77098783e-01]))


In [114]:
print(results_SMM.x)
print(results_SMM.fun)

[ 0.98041205  0.98492512 -0.65024428  0.35622345  0.001     ]
0.244012036971
